# 配置项

In [1]:
# root_id = '20221125094240-jdfn6ce'  # 课程/人工智能与机器学习/网络空间安全领域智能化
root_id = '20220909082530-85ixym4'  # 课程/人工智能与机器学习
path_root = './data/dev'  # 数据目录
path_tree = f'{path_root}/tree/{root_id}.json'  # 节点树文件
path_tree_with_keys = f'{path_root}/tree/{root_id}-keys.json'  # 提取了关键字的节点树文件
path_tree_with_full_keys = f'{path_root}/tree/{root_id}-full-keys.json'  # 继承了上级节点关键字的树文件
path_index = f'{path_root}/index/{root_id}/'  # 索引文件保存目录

In [2]:
import json
import datetime

In [3]:
# 统计时间
def progress(start, current, final):
    now = datetime.datetime.now()
    duration = now - start
    rate = current/final
    print(f"{str(duration):<15} {str(duration * (1-rate) / rate):<15} {current:8}/{final} {rate*100:8.3f}%", end='\r')

# 构造节点树并保存

In [4]:
import pkg.env as env

from pkg.api import API
from pkg.notebook import Notebook, Notebooks
from pkg.client import Client
from pkg.tree import Tree

In [5]:
client = Client(
    token=env.token,
    host=env.host,
    port=env.port,
    ssl=env.ssl,
    proxies=env.proxies,
)

In [6]:
notebooks = client.getNotebooks()

In [7]:
tree = Tree(
    client=client,
    notebooks=notebooks,
)

In [8]:
tree.buildTree(
    root_id,
)

20220909082530-85ixym4: 课程/人工智能与机器学习


In [9]:
with open(path_tree, 'w') as f:
    f.write(json.dumps(tree.__dict__(), indent=2, ensure_ascii=False))

# 提取各节点关键短语并保存

In [10]:
from pkg.extractor import Extractor
from pkg.tree import Tree

In [11]:
with open(path_tree, 'r') as f:
    tree = Tree.fromDict(json.loads(f.read()))
node_count = len(tree)
print('node count:', node_count)

node count: 3521


In [12]:
extractor = Extractor()

/home/lyl/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
start = datetime.datetime.now()
for i, node in enumerate(tree):
    node.data.extractKeys(extractor)
    progress(start, i+1, node_count)

In [14]:
with open(path_tree_with_keys, 'w') as f:
    f.write(json.dumps(tree.__dict__(), indent=2, ensure_ascii=False))

# 各节点继承关键短语并保存

In [15]:
from pkg.tree import Tree

In [16]:
with open(path_tree_with_keys, 'r') as f:
    tree = Tree.fromDict(json.loads(f.read()))
node_count = len(tree)
print('node count:', node_count)

node count: 3521


In [17]:
start = datetime.datetime.now()
for i, node in enumerate(tree):
    node.data.inheritKeys(node.parent.data)
    progress(start, i+1, node_count)

In [18]:
with open(path_tree_with_full_keys, 'w') as f:
    f.write(json.dumps(tree.__dict__(), indent=2, ensure_ascii=False))

# 索引各节点

In [19]:
from pkg.fts import FTS
from pkg.tree import Tree

In [20]:
with open(path_tree_with_full_keys, 'r') as f:
    tree = Tree.fromDict(json.loads(f.read()))
node_count = len(tree)
print('node count:', node_count)

node count: 3521


In [21]:
fts = FTS(
    index_dir=path_index
)
fts.initIndex()
# fts.openIndex()

In [22]:
start = datetime.datetime.now()
for i, node in enumerate(tree):
    fts.add_document(
        id=node.id,
        keys=','.join(node.data.keys_with_inherit),
        content=node.data.content,
    )
    progress(start, i+1, node_count)
fts.commit()

# 查询

In [23]:
from pkg.fts import FTS
from pkg.tree import Tree
from pkg.extractor import Extractor

In [24]:
with open(path_tree_with_full_keys, 'r') as f:
    tree = Tree.fromDict(json.loads(f.read()))
node_count = len(tree)
print('node count:', node_count)

node count: 3521


In [25]:
fts = FTS(
    index_dir=path_index
)
# fts.initIndex()
fts.openIndex()

In [26]:
extractor = Extractor()

In [27]:
extractor.parse({'应用'})

[['TOP', [['IP', [['VP', [['VV', ['应用']]]]]]]]]

In [34]:
# query = '网络空间安全 机器学习 流程'
# query = '网络空间安全及主要研究内容'
# query = '网络空间安全 系统安全'
query = '在网络空间安全领域中机器学习的应用'
phrases, words  = extractor.classify(set(query.split()))

query = f"{' OR '.join(phrases)} OR ({' AND '.join(words)})"
print(phrases, words)
print(query)

{'机器学习的应用', '网络空间安全领域', '机器学习', '应用'} {'网络', '安全', '机器', '应用', '学习', '领域', '空间'}
机器学习的应用 OR 网络空间安全领域 OR 机器学习 OR 应用 OR (网络 AND 安全 AND 机器 AND 应用 AND 学习 AND 领域 AND 空间)


In [35]:
with fts.searcher as s:
    results = s.search(fts.parse(query).query)
    for result in results:
        node_id = result['id']
        breadcrumb = ' > '.join(map(lambda node: node.data.content, tree.getBreadcrumb(node_id)))
        print(f"siyuan://blocks/{node_id}", breadcrumb)

siyuan://blocks/20221223094706-caqg3kr 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > ​image 机器学习在网络空间安全中的典型应用 应用安全 应用软件安全 垃圾邮件检测 基于URL的恶意网页识别 恶意 PDF 检测 社会网络安全 社交网络异常账号检测 信用卡欺诈检测 取证分析 网络舆情 网络安全 网络基础设施安全 BGP异常检测 恶意域名检测 网络安全检测 僵尸网络检测 网络入侵检测技术 恶意加密流量识别 系统安全 芯片安全 劣质芯片检测 硬件木马检测 PUF攻击 系统硬件及物理环境安全 设备身份认证 密码设备边信道攻击 伪基站检测 系统软件安全 漏洞分析与挖掘 恶意代码分析 用户身份认证 虚拟化安全 机器学习的一般应用流程 问题抽象 数据采集 数据预处理及安全提取 模型构建 模型验证 效果评估 assets/image-20221223094706-nm9ch2o.png ​
siyuan://blocks/20230302212920-j19azs0 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > 机器学习的一般应用流程
siyuan://blocks/20230302212920-qrdy1ru 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > 机器学习的一般应用流程 > 模型验证
siyuan://blocks/20230302212920-iku3d52 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > 机器学习的一般应用流程 > 问题抽象
siyuan://blocks/20230302212920-swhocfb 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > 机器学习的一般应用流程 > 数据采集
siyuan://blocks/20230302212920-1rblrhs 课程 > 人工智能与机器学习 > 网络空间安全领域智能化 > 网络空间安全领域的智能化 > 机器学习的一般应用流程 > 模型构建
siyuan://blocks/20230302212920-026s1mz 课程 > 人工智能